In [ ]:
from fastai.imports import *

In [ ]:
from scipy.stats import f_oneway
import pingouin as pg

In [ ]:
df = pd.read_excel('All2021Cohorts_merged_outlier managed_copy_2.xlsx', sheet_name='MergedRTV_Control2021',header=5)

In [ ]:
vsf_data =  pd.read_excel('vsf_safe.xlsx', sheet_name='vsf')

In [ ]:
vsf_data['hhid_2']

In [ ]:
peer = df[df['Status']=='PEER']
rtv = df[df['Status']=='RTV']

In [ ]:
rtv['save_mode_7'].value_counts()

In [ ]:
df['Status'].value_counts()

In [ ]:
df['new_status'] = np.where(df['Status'] == 'PEER', 'PEER', 
                            np.where(df['Status'] == 'RTV', 
                                    np.where(df['save_mode_7'] == 1, 
                                    'RTV_vsla', 'RTV_nonvsla'), 
                                    df['Status']))

In [ ]:
df['new_status'].value_counts()

In [ ]:
rtv_vsf = rtv[rtv["save_mode_7"] == 1]
rtv_nvsf =  rtv[rtv["save_mode_7"]==0]

In [ ]:
rtv_vsf.shape, rtv_nvsf.shape

In [ ]:
vsf_1 = rtv_vsf[rtv_vsf['hhid_2'].isin(vsf_data['hhid_2'])]

In [ ]:
vsf_1['Status'].value_counts()

In [ ]:
rtv_vsf.loc[:, 'state'] = np.where(rtv_vsf['hhid_2'].isin(vsf_data['hhid_2']), 'vsf', 'non vsf')

In [ ]:
rtv_vsf['state'].value_counts()

In [ ]:
adjusted_cpi = 80.66

In [ ]:
peer.shape, rtv_vsf.shape, rtv_nvsf.shape

In [ ]:
rtv_vsf_new = rtv_vsf[rtv_vsf['hhid_2'] != 'MIT-KAI-IDD-M7572']

## Savings / Loans

In [ ]:
# rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'Total Savings (Ugx)'] = rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'saving_amount_rtv_vsla_cashround'].apply(lambda x: x * 0.7 if x > 250000 else x)

In [ ]:
rtv_vsf_new.groupby('state')['Total Savings (Ugx)'].mean()

In [ ]:
## Loan Amount
(rtv_vsf.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))

In [ ]:
subset = rtv_vsf[rtv_vsf['borrowed_past_12_months'] == 1]

In [ ]:
(subset.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))

### Percentage of household active

In [ ]:
non_vsf = rtv_vsf[rtv_vsf["state"] == 'non vsf']
vsf = rtv_vsf[rtv_vsf["state"] == 'vsf']

In [ ]:
non_vsf.shape, vsf.shape

In [ ]:
# helper function
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df][0]
    return name

In [ ]:
for frame in [vsf, non_vsf]:
    percent = (len(frame[frame['Total Savings (Ugx)'] != 0]) / len(frame)) * 100
    print(f"{get_df_name(frame)}:{percent}")

In [ ]:
for frame in [vsf, non_vsf]:
    save = frame.loc[frame['Total Savings (Ugx)'] != 0]['Total Savings (Ugx)'].mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:{save}")

### Total Outstanding debt

In [ ]:
columns = ['bank_loan_remaining',
            'sacco_loan_remaining',
            'shop_keeper_loan_remaining',
            'family_friend_loan_remaining',
            'rtv_vsla_loan_remaining',
            'other_village_VSLA_loan_remaining',
            'money_lender_loan_remaining',
            'other_loan_source_loan_remaining']

In [ ]:
for frame in [vsf, non_vsf]:
    debt = frame[columns].fillna(0).sum(axis=1).mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:\t{debt}")

In [ ]:
# peer_debt = peer[columns].sum(axis=1)
# rtv_vsf_debt = rtv_vsf[columns].sum(axis=1)
# rtv_nvsf_debt = rtv_nvsf[columns].sum(axis=1)

In [ ]:
# f_oneway(peer_debt, rtv_vsf_debt, rtv_nvsf_debt)

### Loan in the first 12 months

In [ ]:
loan = (rtv_vsf.groupby('state')['borrowed_past_12_months'].value_counts(normalize=True)*100).unstack().transpose()
loan

### Interest Rate

In [ ]:
for frame in [vsf, non_vsf]:
    interest = frame[frame['Interest'] != 0.]['Interest'].mean()
    print(f"{get_df_name(frame)}:\t{interest}")

# Income Earnings and Asssets

In [ ]:
rtv_vsf.groupby('state')[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
    'HH Income + Consumption + Residues/Day (USD_Cons_rate)'
]].mean().transpose()

## Purpose of the Loan

In [ ]:
((rtv_vsf.groupby('state')['purpose_for_borrow'].value_counts(normalize=True)*100).unstack()).transpose()

## Loan Sources

In [ ]:
#Bank Account
(rtv_vsf.groupby('state')['Loan_from_1'].value_counts(normalize=True)*100).unstack().transpose()

In [ ]:
#saving and Credit accounts
(rtv_vsf.groupby('state')['Loan_from_2'].value_counts(normalize=True)*100).unstack().transpose()

In [ ]:
# RTV VSLA
(rtv_vsf.groupby('state')['Loan_from_7'].value_counts(normalize=True)*100).unstack().transpose()

In [ ]:
# Other VSLA
(rtv_vsf.groupby('state')['Loan_from_8'].value_counts(normalize=True)*100).unstack().transpose()

## Interests Rates

In [ ]:
(rtv_vsf.groupby('state')[[
        'bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

## Loan Types

In [ ]:
(rtv_vsf.groupby('state')['bank_loan_type'].value_counts(normalize=True) * 100).unstack().transpose()